# IMU Data Parsing
This notebook takes initial steps to parsing the IMU data accompanying the Sunset Park 2019 scan (https://archive.nyu.edu/handle/2451/60458). This work is not utilized in the paper.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
# load the data
columns = ['time',
 'distance',
 'easting',
 'northing',
 'ellipsoid_height',
 'latitude',
 'longitude',
 'ellipsoid_height_1',
 'roll',
 'pitch',
 'heading',
 'east_velocity',
 'north_velocity',
 'up_velocity',
 'east_sd',
 'north_sd',
 'height_sd',
 'roll_sd',
 'pitch_sd',
 'heading_sd']

imu_df = pd.read_csv("../../Data/imu_data/export_Mission_2.txt", 
                     sep='[ ]+',
                     engine='python',
                     skiprows=28,
                     header=None,
                     names=columns)

In [ ]:
# Import anchor pickle
anchors = pd.read_pickle("../../Data/pickles/anchor_points_164239_just_xyz.pkl")
anchors_df = pd.DataFrame(anchors,columns=['gps_time'])

In [ ]:
def slice_imu(imu_df,lidar_df,imu_time_name = 'time',lidar_time_name='gps_time'):
    #Function slices imu_df for the points that fall within the GPS timeframe of the provided lidar_df.
    
    # Determine gps timeframe for lidar
    lidar_range = (lidar_df[lidar_time_name].min(),lidar_df[lidar_time_name].max())
    print("Timespan of LiDAR data: {:2.2f} seconds".format(lidar_range[1]-lidar_range[0]))
    
    # Slide of IMU data within lidar_range
    in_slice = (imu_df[imu_time_name]>=lidar_range[0]) & (imu_df[imu_time_name]<=lidar_range[1])
    imu_slice = imu_df[in_slice]
    print("IMU points in timespan: {:d}".format(imu_slice.shape[0]))
    return imu_slice

In [ ]:
# Create IMU slice
imu_slice = slice_imu(imu_df,anchors_df)

In [ ]:
%time
a = np.zeros(len(anchors_df))
for i,time in enumerate(imu_slice['time']):
    a = [time if time <= pt else a[i] for i,pt in enumerate(anchors_df['gps_time'])]
    if i%10 == 0:
        print("Iteration count: ",i)
    
anchors_df['imu_time'] = a

In [ ]:
# Plot some IMU data
keys = ['time','distance','latitude','longitude','ellipsoid_height','roll','pitch','heading','up_velocity']
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }

plt.figure(figsize=(20,25))
for i, key in enumerate(keys):
    plt.subplot(3,3, (i+1))
    plt.plot(imu_slice['time'],imu_slice[key],'xr')
    plt.title(str(key),fontdict=font)
    if i >= 6:
        plt.xlabel("GPS Time (s)",fontdict=font)

In [ ]:
field = 'pitch'
plt.figure(figsize=(20,25))
plt.plot(imu_slice['time'],imu_slice[field],'xr')
plt.ylabel(field)
plt.xlabel("GPS Time")

### Find a specific Lat/Lon
40°38'38.8"N 74°01'33.0"W

In [ ]:
def convert_lat_lon_to_dec(degrees,minutes,seconds):
    return degrees + (minutes/60) + (seconds/3600)
lat = convert_lat_lon_to_dec(40,38,38.8)
lon = convert_lat_lon_to_dec(74,1,33.0)*-1
print("Latitude: {:2.4f}".format(lat))
print("Longitude: {:2.4f}".format(lon))

In [ ]:
# Sort by nearest lat-lon ()
np.argmin(abs(imu_df['latitude']+imu_df['longitude']-lat-lon))

In [ ]:
imu_df['latitude'][222005]

In [ ]:
imu_df['longitude'][222005]

In [ ]:
imu_df.head()